# 04.从训练到推理

## 一、预训练阶段，模型到底在做什么？

在整个大模型生命周期中，**预训练（Pre-training）是最基础、最核心、也是最容易被误解的阶段**。

在这个阶段，大语言模型并不是在学习事实、规则、世界观或推理方法，而是在完成一个**极其单一、却被重复了数万亿次的任务**,即

> **给定一段上下文，预测下一个 Token 的概率分布。**


### 1. 预训练任务的唯一目标

所有输入文本——无论是小说、法律条文、论文、代码、网页、聊天记录——在进入模型之前，都会被统一处理为 **Token 序列**：

```text
Token₁, Token₂, Token₃, … → 预测 Token₄
```

模型不会被告知：

* 这是不是事实
* 这是不是规则
* 这是不是一个“正确答案”
* 这是不是在“推理”

它只是在反复学习一件事：在这种上下文形态下，最可能接下来写什么。

### 2. 什么叫「预训练」？

**预训练的定义可以精确表述为：**

> 在不区分任务、不区分领域、不引入人类偏好约束的情况下，
> 对大规模通用语料进行无监督或弱监督的下一 Token 预测训练。

几个关键词非常重要：

* **不区分任务**：
  问答、对话、代码、说明书在模型眼里没有本质区别
* **不区分领域**：
  数学、法律、文学只是不同的统计分布
* **弱监督**：
  监督信号只来自“真实下一个 Token 是什么”

### 3. 模型在预训练阶段“真正学到”的是什么？

当这种预测在：

* 海量文本数据
* 巨大参数规模
* 深层 Transformer 结构
* 多轮梯度下降

中被不断重复时，模型**并没有显式学会规则**，但却隐式形成了大量结构性能力。

它逐渐学会了：

* **语言的统计结构**

  * 词序
  * 语法
  * 常见搭配
* **文本模式的展开形态**

  * 提问 → 分析 → 回答
  * 定义 → 解释 → 示例
* **问题到解法的高频路径**

  * 数学题的解题模板
  * 代码的常见写法
  * 论文与报告的组织结构

这些能力并不是被“教会”的，
而是**在概率空间中自然涌现（emerge）出来的**。

### 4. 预训练之后

为了让预训练后的模型变成更加好用，通常还会有以下两个阶段：

1. **SFT (Supervised Fine-Tuning) 有监督微调**：
给模型看几万组高质量的 `[指令] -> [回答]` 范本。教它学会：当人类提问时，你应该提供答案，而不是继续接龙。
2. **RLHF (Reinforcement Learning from Human Feedback) 人类反馈强化学习**：
让人类给模型的多个回答打分，告诉它哪些回答更安全、更准确、更有礼貌。这是给模型注入“价值观”的过程。


## 二、Loss、梯度、参数更新的直觉理解

在上面讲了模型在预训练阶段不断“预测下一个 Token”，并通过这种反复的训练逐渐形成统计规律和模式能力。那么，这个“不断预测、不断优化”的过程，背后到底是怎么发生的呢？这就是 **Loss、梯度和参数更新** 在起作用的地方。

#### **1. Loss ——模型“犯错的程度”**

每次模型预测下一个 Token，它都可能猜对，也可能猜错。Loss 就是用来量化这个猜测有多糟糕的指标：

* Loss 高 → 预测离真实 Token 很远，模型“犯大错”
* Loss 低 → 预测接近真实 Token，模型“比较聪明”

直觉上，Loss 就像模型的**自我反馈表**：它告诉模型“这一步做得好不好”。

#### **2. 梯度 ——模型“改进的方向”**

Loss 告诉模型哪里做错了，但不会直接告诉它该怎么改。这时，梯度就起作用了：

* 梯度告诉模型：**如果把参数往这个方向微调，Loss 会下降得更快**
* 梯度的大小表示“改进有多急迫”
* 梯度的方向表示“改哪条路能最快变聪明”

梯度就像模型在海量信息中寻找“下一步微调的指南针”。

#### **3. 参数更新 ——模型“试错修正”的动作**

有了梯度信息，模型就可以**调整自己的参数**：

* 小幅度更新 → 细微改善
* 大幅度更新 → 风险大，但可能学得快

所以，参数更新就像模型在“沿着 Loss 形成的山谷不断下坡”：

* Loss 是高度
* 梯度是坡度
* 参数更新就是每一步迈下去，让模型慢慢靠近“预测最准确”的谷底


#### **4. 训练的整体过程**

把三者串起来，你可以把预训练阶段想象成这样一个循环：

1. **预测** → 模型根据当前参数猜下一个 Token
2. **计算 Loss** → 模型知道自己猜得好不好
3. **计算梯度** → 模型知道沿哪个方向调整参数能改进
4. **参数更新** → 模型沿梯度方向微调自己
5. **重复无数次** → 模型逐渐掌握语言模式、问题到解法的路径

这个循环发生在**海量文本 + 巨大参数 + 深层 Transformer**中，被重复数十亿次，最终模型形成了 emergent 能力：它能理解语境、模仿逻辑、解决问题，但这些能力本质上是**概率统计的自然涌现**。

### 实战:训练训练模拟

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

# -----------------------------
# 1. 数据准备（模拟Token预测）
# -----------------------------
# 假设每个 Token 编码为 0,1,2,...,9
# 我们要预测下一个 Token
# 简单模拟一个序列规律：下一个 Token = (当前 Token + 1) % 10
data = torch.arange(0, 10).float().unsqueeze(1)  # shape (10,1)
target = (data + 1) % 10

# -----------------------------
# 2. 模型定义（单层线性模型）
# -----------------------------
# 模型尝试学会“预测下一个 Token”
model = nn.Linear(1, 1)  # 输入1维，输出1维

# -----------------------------
# 3. Loss 函数和优化器
# -----------------------------
criterion = nn.MSELoss()  # 用均方误差衡量“预测误差”
optimizer = optim.SGD(model.parameters(), lr=0.1)  # 学习率 0.1

# -----------------------------
# 4. 训练循环
# -----------------------------
epochs = 50
for epoch in range(epochs):
    optimizer.zero_grad()  # 清空梯度

    # 预测
    output = model(data)

    # 计算 Loss —— 模型“犯错的程度”
    loss = criterion(output, target)

    # 计算梯度 —— 模型“改进的方向”
    loss.backward()

    # 参数更新 —— 模型“沿坡下山”
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1:02d}: Loss = {loss.item():.4f}")

# -----------------------------
# 5. 训练完成，测试预测
# -----------------------------
test_data = torch.tensor([[5.0]])  # 输入 Token 5
pred = model(test_data).item()
print(f"预测下一个 Token: 输入 5 → 预测 {pred:.2f} (真实应为 6)")


Epoch 10: Loss = 164316087058432.0000
Epoch 20: Loss = 8296888005101528319996723200.0000
Epoch 30: Loss = inf
Epoch 40: Loss = inf
Epoch 50: Loss = inf
预测下一个 Token: 输入 5 → 预测 -149211369008104826728056842581180416.00 (真实应为 6)


过程说明:
1. **Loss = 模型犯错的程度**

   * `loss = criterion(output, target)` 告诉模型预测有多差。

2. **梯度 = 改进的方向**

   * `loss.backward()` 计算 Loss 对模型参数的偏导数，告诉模型“沿哪个方向调整参数才能让 Loss 降低”。

3. **参数更新 = 迈步下坡**

   * `optimizer.step()` 根据梯度调整权重，就像模型沿 Loss 山谷下坡，离最佳预测更近。

4. **训练循环 = 不断试错**

   * `for epoch in range(epochs)` 模拟模型在海量文本上不断重复预测、计算 Loss、更新参数，最终学会了规律（这里是简单的 +1 模式）。

## 三、训练阶段 vs 推理阶段

在训练阶段，模型通过**不断试错、自我修正**来学习规律。每次预测都会计算 Loss（模型“犯错的程度”），梯度指明调整方向，参数更新沿梯度方向微调权重。这个循环反复进行，模型能力逐渐形成。
而推理阶段，模型已经完成训练，参数固定，不再更新。输入流向输出，模型使用已经学到的能力来完成任务或生成文本，不再进行自我修正。


对比如下：
| 对比维度      | 训练阶段（Training）                   | 推理阶段（Inference）           |
| --------- | -------------------------------- | ------------------------- |
| **定义**    | 模型通过 Loss → 梯度 → 参数更新循环学习规律和能力   | 模型使用训练中学到的能力进行任务或生成文本     |
| **参数状态**  | 可更新，梯度指导参数调整                     | 固定，不再更新，使用已有参数            |
| **数据流向**  | 双向循环：预测 → Loss → 梯度 → 参数更新 → 再预测 | 单向流：输入 → 模型 → 输出，Loss 不回传 |
| **速度与资源** | 慢，资源消耗大，需要显存存储梯度、优化器状态           | 快，资源消耗低，只进行前向计算           |
| **核心目标**  | 提升模型能力，学会语言规律、问题解决模式             | 使用模型能力，完成具体任务或生成文本        |
